### Figures for the scaling relations meeting at Queen's University in Kingston, ON in 2018 July.

In [1]:
import os, pdb
import numpy as np
import numpy.ma as ma
import pandas as pd
import warnings
from scipy.stats import binned_statistic
from matplotlib.ticker import FormatStrFormatter, MultipleLocator, ScalarFormatter
import matplotlib.pyplot as plt

In [2]:
from astropy.io import fits, ascii
from astropy.table import Table

In [3]:
import seaborn as sns
sns.set(context='talk', style='ticks', font_scale=1.5)#, palette='deep')
sns.set_palette('deep', 12)
colors = sns.color_palette()

In [4]:
import legacyhalos.io

In [5]:
%matplotlib inline

In [6]:
talkdir = os.path.join(os.getenv('HOME'), 'research', 'talks', 
                       'legacyhalos', 'scaling-relations-2018jul')
hdefault = 0.7 # 0.677 # Hubble constant
omega_b = 0.048 # from Knebe+18
omega_m = 0.307 

### Some utility functions.

In [7]:
def mstar_label():
    return r'$log_{10}\, (\mathcal{M}_{*}\ /\ h^{-2}\, \mathcal{M}_{\odot})$'

In [8]:
def mhalo_label():
    return r'$log_{10}\, (\mathcal{M}_{\rm halo}\, /\, \mathcal{M}_{\odot})$'

In [9]:
def mstar_label2():
    return r'$log_{10}\, (\mathcal{M}_{*}\ /\ h^{-1}\, \mathcal{M}_{\odot})$'

In [10]:
def mstar_label3():
    return r'$log_{10}\, (\mathcal{M}_{*}\, /\, \mathcal{M}_{\odot})$'

In [11]:
def phi_label():
    return r'$log_{10}\, (\mathcal{\Phi}\ /\ h^{3}$ Mpc$^{-3}$ dex$^{-1})$'

In [12]:
def mstar_mhalo_label():
    return r'$log_{10}\, (\mathcal{M}_{*}\ /\ \mathcal{M}_{\rm halo})\ /\ (\Omega_b\ /\ \Omega_m)$'

### Ellipse-fitting results

In [13]:
from legacyhalos.io import analysis_dir, read_ellipsefit, read_multiband, read_sky_ellipsefit
from legacyhalos.qa import display_multiband, display_ellipsefit, _display_ellipse_sbprofile

In [14]:
objid, objdir = '0003497', os.path.join(analysis_dir(), '0003497')
ellipsefit = read_ellipsefit(objid, objdir)
skyellipsefit = read_sky_ellipsefit(objid, objdir)
data = read_multiband(objid, objdir)

In [15]:
sbprofilefile = os.path.join(talkdir, '{}-ellipse-sbprofile.png'.format(objid))
_display_ellipse_sbprofile(ellipsefit, skyellipsefit=skyellipsefit, minerr=0.0,
                           png=sbprofilefile, verbose=True, use_ylim=(18, 33))

Writing /Users/ioannis/research/talks/legacyhalos/scaling-relations-2018jul/0003497-ellipse-sbprofile.png


In [16]:
multibandfile = os.path.join(talkdir, '{}-ellipse-multiband.png'.format(objid))
display_multiband(data, ellipsefit=ellipsefit, png=multibandfile, verbose=True, vertical=True)

Writing /Users/ioannis/research/talks/legacyhalos/scaling-relations-2018jul/0003497-ellipse-multiband.png


In [17]:
stop

NameError: name 'stop' is not defined

### Toy stellar mass function and SMHM relation

In [ ]:
def qa_smf(png=None):
    """A toy SMF."""
    
    # Halo mass function from Tinker's code 
    #   http://cosmo.nyu.edu/~tinker/massfunction/mf.html
    # with ./massfunction.x mf-toy.bat
    
    mh, dndm = np.loadtxt(os.path.join(talkdir, 'toy.dndM'), unpack=True)
    mh, dndm = np.log10(mh), np.log10(dndm)
    
    # SMF
    mstarlim = (9, 12)
    mstar = np.linspace(8, 12, 50)
    alpha = -1.2
    mstar0 = 10.7
    
    phi = np.zeros_like(mstar)
    phi += (alpha + 1) * (mstar - mstar0) * np.log(10)
    phi -= pow(10, mstar - mstar0)
    #phierr = np.polyval([0.05, 0.25], mstar-mstar0)
    #phierr[phierr < 0.1] = 0.1
    phierr = 0.12*np.abs(phi)
    phierr[phierr < 0.05] = 0.05
    
    fig, ax = plt.subplots(figsize=(10, 7))
    #keep = (mstar > 9.1) * (phi > -4.95)
    #ax.fill_between(mstar[keep], phi[keep]-0.2, phi[keep]+0.2, alpha=0.7)
    ax.fill_between(mstar, phi+phierr, phi-phierr, alpha=0.7,
                    label='Stellar Mass Function')
    
    ax.plot(mh-4, dndm+19.3, color=colors[1], lw=4, ls='--',
            label='Halo Mass Function')
    
    #ax.plot(mstar, phi)
    ax.grid()
    #ax.set_xlim(mstarlim)
    #ax.set_ylim(-5, 1.5)
    ax.set_xlim(9, 12)
    ax.set_ylim(-7, 3)
    
    #ax.set_yscale('log')
    
    #ax.set_yticks([0.004, 0.01, 0.03, 0.1, 0.3])
    #ax.get_yaxis().set_major_formatter(ScalarFormatter())

    ax.set_xlabel(mstar_label3())
    ax.set_ylabel(r'$\log_{10}$ (Number Density) (arbitrary units)')
    
    ax.legend(loc='upper right')
    
    if png:
        plt.savefig(os.path.join(talkdir, png))

In [ ]:
qa_smf(png='toy-smf.png')

In [ ]:
def qa_smhm_relation(png=None):
    """Data taken from Behroozi+13."""
    mhalo, mstar_mhalo, errlo, errup = np.loadtxt(os.path.join(
        os.getenv('CATALOGS_DIR'), '13behroozi',  'smmr', 
        'c_smmr_z0.10_red_all_smf_m1p1s1_bolshoi_fullcosmos_ms.dat'),
                                              unpack=True)
    keep = mhalo > 11.2
    mstar_mhalo = mstar_mhalo[keep]
    mhalo = mhalo[keep]
    errlo = errlo[keep]
    errup = errup[keep]
    
    sfe = 10**(mstar_mhalo + np.log10(omega_m / omega_b))
    sfe_up = 10**(mstar_mhalo + errup + np.log10(omega_m / omega_b))
    sfe_lo = 10**(mstar_mhalo - errlo + np.log10(omega_m / omega_b))
    keep = sfe > 0.004

    fig, ax = plt.subplots(figsize=(10, 7))
    ax.fill_between(mhalo[keep], sfe_lo[keep], sfe_up[keep], alpha=0.7)
    #ax.plot(mhalo, sfe)
    ax.grid()
    ax.set_xlim(11, 15)
    ax.set_ylim(0.003, 0.4)
    ax.set_yscale('log')
    
    ax.set_yticks([0.004, 0.01, 0.03, 0.1, 0.3])
    ax.get_yaxis().set_major_formatter(ScalarFormatter())

    ax.set_xlabel(mhalo_label())
    ax.set_ylabel(mstar_mhalo_label())
    
    ax.text(11.2, 0.006, 'Central Galaxies', va='center', 
            ha='left', fontsize=20, style='italic')
    
    if png:
        plt.savefig(os.path.join(talkdir, png))    

In [ ]:
qa_smhm_relation(png='toy-smhm.png')

### Model stellar mass-halo mass relation

Data taken from Knebe+18.

In [ ]:
def read_carnage_smhm(fullrange=False, seed=None, h=hdefault):
    """Data taken from Knebe+18."""
    
    carnagedir = os.path.join(os.getenv('CATALOGS_DIR'), 'carnage')
    
    labels = ('DLB07', 'GalICS2.0', 'LGALAXIES', 'SAG', 'MICE', 
              'SAGE', 'Galform-GP14', 'MORGANA', 'ySAM')    
    data = []
    for model in ('DLB07-c01.txt', 'GalICS2.0-c01.txt', 'LGALAXIES-c01.txt',
                  'SAG-c01.txt', 'MICE-c01.txt', 'SAGE-c01.txt', 
                  'Galform-GP14-c01.txt', 'MORGANA-c01.txt', 'ySAM-c01.txt'):
        dd = ascii.read(os.path.join(carnagedir, 'Fig.11a', model), 
                        format='basic', names=('mhalo', 'mstar_mhalo', 'junk1', 
                                               'junk2', 'junk3', 'junk4'),
                        exclude_names=('junk1', 'junk2', 'junk3', 'junk4'))
        #good = np.where( np.isfinite(dd['phi']) * (dd['mstar'] > 0) )[0]
        #dd = dd[good]
        
        dd['mhalo'] = np.log10(dd['mhalo'] / h)
        dd['mstar_mhalo'] = dd['mstar_mhalo'] * omega_m / omega_b
        #dd['mstar_mhalo'] = np.log10(dd['mstar_mhalo'])
        data.append(dd)
        
    if fullrange:
        rand = np.random.RandomState(seed)
        
        mhalo, mstar_mhalo = [], []
        for dd in data:
            #good = np.where( np.isfinite(dd['phi']) * (dd['mstar'] > 0) )[0]
            mhalo.append(dd['mhalo'] + rand.normal(loc=0, scale=0.2))
            mstar_mhalo.append(dd['mstar_mhalo'])
        mhalo = np.hstack(mhalo)
        mstar_mhalo = np.hstack(mstar_mhalo)
        #print(mstar_mhalo)
        
        width = 0.5
        mhalorange = (10, 15.5)
        nbins = np.int(np.ptp(mhalorange) / width)
        
        mstar_mhalomin, mhalo_edges, _ = binned_statistic(mhalo, mstar_mhalo, statistic='min', 
                                                          bins=nbins, range=mhalorange)
        mstar_mhalomax, _, _ = binned_statistic(mhalo, mstar_mhalo, statistic='max', 
                                                bins=nbins, range=mhalorange)
        mhalo = mhalo_edges[1:] - width / 2
        
        good = np.isfinite(mhalo) * np.isfinite(mstar_mhalomin) * np.isfinite(mstar_mhalomax)
        return mhalo[good], mstar_mhalomin[good], mstar_mhalomax[good]
    
    else:
        return data, labels

In [ ]:
def qa_carnage_smhm(individual=True, fullrange=False, png=None, ylim=None):
    
    fig, ax = plt.subplots(figsize=(10, 7))
    if individual:
        data, labels = read_carnage_smhm()
        ls = ('-', '--', '-.', ':')
        for dd, ll in zip(data, labels):
            ax.plot(dd['mhalo'], dd['mstar_mhalo'], label=ll, lw=4,
                    ls=ls[0], alpha=0.8)
            ls = np.roll(ls, 1)

    if fullrange:
        mhalo, mstar_mhalomin, mstar_mhalomax = read_carnage_smhm(fullrange=True, seed=1)
        #print(mhalo, mstar_mhalomin, mstar_mhalomax)
        ax.fill_between(mhalo, mstar_mhalomin, mstar_mhalomax, alpha=0.2, color='gray',
                       label='All CARNage Models')

    ax.grid()
    ax.set_xlim(11, 15)
    ax.set_ylim(0.003, 0.4)
    ax.set_yscale('log')
    
    ax.set_yticks([0.004, 0.01, 0.03, 0.1, 0.3])
    ax.get_yaxis().set_major_formatter(ScalarFormatter())

    ax.set_xlabel(mhalo_label())
    ax.set_ylabel(mstar_mhalo_label())
    ax.legend(loc='lower left', ncol=2, fontsize=16)
    
    plt.subplots_adjust(left=0.15, right=0.97, bottom=0.16, top=0.95)
    
    if png:
        plt.savefig(os.path.join(talkdir, png))

In [ ]:
qa_carnage_smhm(fullrange=True, png='smhm-carnage-fullrange.png')

In [ ]:
qa_carnage_smhm(png='smhm-carnage.png')

### Model stellar mass functions

Data taken from Knebe+18.

In [ ]:
def read_carnage_smf(fullrange=False, seed=None, h=hdefault):
    """Data taken from Knebe+18."""
    
    carnagedir = os.path.join(os.getenv('CATALOGS_DIR'), 'carnage')
    
    labels = ('DLB07', 'GalICS2.0', 'LGALAXIES', 'SAG', 'MICE', 
              'SAGE', 'Galform-GP14', 'MORGANA', 'ySAM')    
    data = []
    for model in ('DLB07-c01.txt', 'GalICS2.0-c01.txt', 'LGALAXIES-c01.txt',
                  'SAG-c01.txt', 'MICE-c01.txt', 'SAGE-c01.txt', 
                  'Galform-GP14-c01.txt', 'MORGANA-c01.txt', 'ySAM-c01.txt'):
        dd = ascii.read(os.path.join(carnagedir, 'Fig.1', model), 
                        format='basic', names=('mstar', 'phi', 'junk1', 'junk2', 'junk3'),
                        exclude_names=('junk1', 'junk2', 'junk3'))
        good = np.where( np.isfinite(dd['phi']) * (dd['mstar'] > 0) )[0]
        dd = dd[good]
        
        dd['mstar'] = np.log10(dd['mstar'] / h)
        dd['phi'] = np.log10(dd['phi'] * h**3)
        data.append(dd)
        
    if fullrange:
        rand = np.random.RandomState(seed)
        
        mstar, phi = [], []
        for dd in data:
            #good = np.where( np.isfinite(dd['phi']) * (dd['mstar'] > 0) )[0]
            mstar.append(dd['mstar']+ rand.normal(loc=0, scale=0.05))
            phi.append(dd['phi'])
        mstar = np.hstack(mstar)
        phi = np.hstack(phi)
        
        width = 0.2
        mstarrange = (9.1, 12.5)
        nbins = np.int(np.ptp(mstarrange) / width)
        
        phimin, mstar_edges, _ = binned_statistic(mstar, phi, statistic='min', bins=nbins, 
                                                  range=mstarrange)
        phimax, _, _ = binned_statistic(mstar, phi, statistic='max', bins=nbins,
                                                  range=mstarrange)
        mstar = mstar_edges[1:] - width / 2
        
        return mstar, phimin, phimax
    else:
        return data, labels

In [ ]:
def qa_carnage_smf(individual=True, fullrange=False, png=None, ylim=None):
    fig, ax = plt.subplots(figsize=(10, 7))
    if individual:
        data, labels = read_carnage_smf()
        ls = ('-', '--', '-.', ':')
        for dd, ll in zip(data, labels):
            ax.plot(dd['mstar'], dd['phi'], label=ll, lw=4,
                    ls=ls[0], alpha=0.8)
            ls = np.roll(ls, 1)

    if fullrange:
        mstar, phimin, phimax = read_carnage_smf(fullrange=True, seed=1)
        ax.fill_between(mstar, phimin, phimax, alpha=0.2, color='gray',
                       label='All CARNage Models')

    ax.grid()
    ax.set_xlim(9, 12.5)
    if ylim:
        ax.set_ylim(ylim)

    ax.set_xlabel(mstar_label2())
    ax.set_ylabel(phi_label())
    ax.legend(loc='lower left', ncol=2, fontsize=16)
    
    if png:
        plt.savefig(os.path.join(talkdir, png))

In [ ]:
qa_carnage_smf(fullrange=True)

### Observed stellar mass functions

Data taken from Huang+18.

In [ ]:
def read_moustakas13(starforming=False, quiescent=False):
    """Read data from Moustakas+13."""

    moosedir = os.path.join(os.getenv('CATALOGS_DIR'), '13moustakas')
    
    suffix = 'all'
    if starforming:
        suffix = 'starforming'
    if quiescent:
        suffix = 'quiescent'
    smffile = os.path.join(moosedir, 'smf_{}_supergrid01.txt'.format(suffix))

    data = ascii.read(smffile, format='basic', names=('zlo', 'zhi', 'ngal', 'mstar', 
                                                      'limit', 'phi', 'philo', 'phihi', 
                                                      'phicv'))
    loz = np.where( data['zhi'] <= 0.2 )[0]
    data = data[loz]
    
    return data['mstar'], data['phi'], data['phi'] - data['phicv'], data['phi'] + data['phicv']

In [ ]:
def read_bernardi17(h=hdefault, dustfree=True):
    """Read data from Bernardi+17."""
    moosedir = os.path.join(os.getenv('CATALOGS_DIR'), '17bernardi')
    data = pd.read_table(os.path.join(moosedir, 'smf.txt'), 
                         delim_whitespace=True)

    mstar = data['mstar'].values # - np.log10(h**2)
    if dustfree:
        phi = data['phi_M14_dustfree_obs'].values # + np.log10(h**3)
        philo = phi - data['phierr_M14_dustfree'].values
        phihi = phi + data['phierr_M14_dustfree'].values
    else:
        phi = data['phi_M14_dust_obs'].values # + np.log10(h**3)
        philo = phi - data['phierr_M14_dust'].values
        phihi = phi + data['phierr_M14_dust'].values
    
    return mstar, phi, philo, phihi

In [ ]:
def read_huang18(cModel=False, BCG=False, total=False, h=hdefault):
    
    huangdir = os.path.join(os.getenv('CATALOGS_DIR'), '18huang')

    if BCG:
        censuffix = 'cenhighmh'
    else:
        censuffix = 'cenlowmh'
        
    if cModel:
        fluxsuffix = 'cmodel'
    else:
        fluxsuffix = '100kpc'

    if total:
        data = Table.read(os.path.join(huangdir, 's16a_massive_{}_{}_smf.fits'.format('cenhighmh', fluxsuffix)))
        _data = Table.read(os.path.join(huangdir, 's16a_massive_{}_{}_smf.fits'.format('cenlowmh', fluxsuffix)))
        #print(data['logm_mean'], data['smf'].data, _data['smf'].data)
        
        data['smf'] += _data['smf']
        data['smf_err'] = np.sqrt( data['smf_err']**2 + _data['smf_err']**2 )
        data['smf_low'] = np.sqrt( data['smf_low']**2 + _data['smf_low']**2 )
        data['smf_upp'] = np.sqrt( data['smf_upp']**2 + _data['smf_upp']**2 )
    else:
        smffile = os.path.join(huangdir, 's16a_massive_{}_{}_smf.fits'.format(censuffix, fluxsuffix))
        data = Table.read(smffile)
    
    good = np.where( (data['logm_0'] >= 11.4) * (data['smf_err'] > 0) )[0]
    mstar = data['logm_mean'][good] # - 2*np.log10(h)
    phi = np.log10(data['smf'][good]) # * h**3)
    phierr = data['smf_err'][good] / data['smf'][good] / np.log(10)
    philo = phi - data['smf_low'][good] / data['smf'][good] / np.log(10)
    phihi = phi + data['smf_upp'][good] / data['smf'][good] / np.log(10)
    
    return mstar, phi, philo, phihi

In [ ]:
def double_schechter(logmass, logmstar1=10.0, logmstar2=11.0, 
                     alpha1=-1, alpha2=-1.1, phistar1=0.008, 
                     phistar2=0.001):
    
    mratio1 = pow(10, logmass - logmstar1)
    mratio2 = pow(10, logmass - logmstar2)
    model = np.log(10) * ( np.exp(-mratio1) * phistar1 * mratio1**(alpha1 + 1) + 
                           np.exp(-mratio2) * phistar2 * mratio2**(alpha2+1) )
    
    return model

def read_dsouza15(h=hdefault):
    logmass = np.arange(9.5, 12.5, 0.05)
    
    phimodel = double_schechter(logmass, logmstar1=10.615, logmstar2=10.995,
                                alpha1=-1.082, alpha2=-1.120, 
                                phistar1=0.008579, phistar2=0.000355)
    
    logmass = logmass - np.log10(h**2)
    logphi = np.log10(phimodel * h**3)
    good = np.where(logphi > -7.6)[0]
    
    return logmass[good], logphi[good]

In [ ]:
def qa_smf(plot_carnage=True, plot_huang=True, png=None):
    fig, ax = plt.subplots(figsize=(10, 7))

    # Model shaded region
    if plot_carnage:
        mstar, phimin, phimax = read_carnage_smf(fullrange=True, seed=1)
        ax.fill_between(mstar, phimin, phimax, alpha=0.2, color='gray', 
                        label='CARNage Models (z~0)')

    # Moustakas+13
    mstar, phi, philo, phihi = read_moustakas13()
    ax.fill_between(mstar, philo, phihi, alpha=0.3)
    ax.plot(mstar, phi, lw=3, ls='-', label='Moustakas+13 (z~0.1)')
    
    # D'Souza+15
    mstar, phi = read_dsouza15()
    ax.fill_between(mstar, phi, phi, alpha=0.3)
    ax.plot(mstar, phi, lw=3, ls='-', label="D'Souza+15 (z~0.1)")
    
    # Bernardi+17
    mstar, phi, philo, phihi = read_bernardi17(dustfree=True)
    ax.fill_between(mstar, philo, phihi, alpha=0.3)
    ax.plot(mstar, phi, lw=3, ls='-', label='Bernardi+17 (z~0.1)')
    
    # Huang+18
    if plot_huang:
        mstar, phi, philo, phihi = read_huang18(cModel=False, BCG=False, total=True)
        ax.fill_between(mstar, philo, phihi, alpha=0.3)
        ax.plot(mstar, phi, lw=3, ls='-', label='Huang+18 (z~0.4)')# (HSC, <100 kpc)')

    ax.set_xlabel(mstar_label3())
    ax.set_ylabel(phi_label())
    
    ax.set_ylim(-7.7, -1.5)
    ax.set_xlim(9, 12.5)
    
    ax.grid()
    hh, ll = ax.get_legend_handles_labels()
    #handles = handles[np.array(list(1, 2, 0
    #labels = labels[np.array([1, 2, 0])]
    ax.legend(loc='lower left', fontsize=18)
    #ax.legend([hh[0],hh[1],hh[2],hh[3]], [ll[0],ll[1],ll[2]], loc='lower left', fontsize=18)
    
    if png:
        plt.savefig(png)    

In [ ]:
qa_smf(plot_carnage=True, plot_huang=False, png=os.path.join(talkdir, 'smf-data-carnage-nohuang.png'))

In [ ]:
def qa_smf_huang18(minmax=False, png=None):

    def plot_huang18(ax, cModel=False, BCG=False, label=''):
        mstar, phi, philo, phihi = read_huang18(cModel=cModel, BCG=BCG)
        ax.fill_between(mstar, philo, phihi, alpha=0.3)
        ax.plot(mstar, phi, label=label, lw=3, ls='-')
    
    fig, ax = plt.subplots(figsize=(10, 7))

    # Model shaded region
    # mstar, phimin, phimax = read_model_smf(minmax=True, seed=1)
    # ax.fill_between(mstar, phimin, phimax, alpha=0.2, color='gray')

    # Data
    plot_huang18(ax, cModel=False, BCG=False, 
                 label=r'$\log\,(\mathcal{M}_h/\mathcal{M}_{\odot})<14.2$')#, $R<100$ kpc')
    #plot_huang18(ax, cModel=True, BCG=False, label=r'$\log\,(M_h/M_{\odot})<14.2$, cModel')
    plot_huang18(ax, cModel=False, BCG=True, 
                 label=r'$\log\,(\mathcal{M}_h/\mathcal{M}_{\odot})>14.2$')#, R<100 kpc$')
    #plot_huang18(ax, cModel=True, BCG=True, label=r'$\log\,(M_h/M_{\odot})>14.2$, cModel')

    ax.set_xlabel(mstar_label())
    ax.set_ylabel(phi_label())
    #ax.set_xlim(11.4, 12.3)
    
    ax.grid()
    ax.legend(loc='upper right', ncol=1, fontsize=18)
    
    if png:
        plt.savefig(png)

In [ ]:
qa_smf_huang18()

### Richness vs redshift for the full sample

In [ ]:
legacyhalos_dir = os.getenv('LEGACYHALOS_DIR')
parentfile = os.path.join(legacyhalos_dir, 'legacyhalos-parent.fits')

In [ ]:
rm = Table(fits.getdata(parentfile, extname='REDMAPPER'))

In [ ]:
def qa_legacyhalos_sample(png=None):
    fig, ax = plt.subplots(figsize=(8, 6))

    hb = ax.hexbin(rm['Z'], np.log10(rm['LAMBDA_CHISQ']), mincnt=3,
                   cmap=plt.cm.get_cmap('RdYlBu'), alpha=0.5, bins='log')
    ax.set_xlabel('Redshift $z$')
    ax.set_ylabel(r'$\log_{10}$ (Richness $\lambda$)')
    ax.set_xlim(0, 0.6)
    ymin, ymax = np.log10(3), 2.1
    ax.set_ylim(ymin, ymax)
    ax.axhline(y=np.log10(5), ls='--', color='k')

    cax = fig.add_axes([0.1, 1.05, 0.8, 0.05])
    cb = plt.colorbar(hb, orientation='horizontal', cax=cax)
    cb.set_label(r'$\log_{10}$ (Number of Galaxies)')       

    ax2 = ax.twinx()
    ax2.set_ylabel(r'$\log_{10}\, (\mathcal{M}_{200}\, /\, \mathcal{M}_{\odot})$ at $z=0.3$')
    ax2.set_ylim( np.log10(lambda2mhalo(10**ymin, redshift=0.3)), 
                 np.log10(lambda2mhalo(10**ymax, redshift=0.3)) )
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))    
    ax2.plot([],[])    
    
    if png:
        plt.savefig(png, bbox_inches='tight')

In [ ]:
qa_legacyhalos_sample(png=os.path.join(talkdir, 'legacyhalos-sample.png'))

In [ ]:
stop

### SMHM relation

In [ ]:
cat = legacyhalos.io.read_catalog(extname='LSPHOT-ISEDFIT', upenn=False, 
                                   isedfit=True, columns=('mstar_avg', 'sfr100_avg'))
cat1 = legacyhalos.io.read_catalog(extname='REDMAPPER', upenn=False, 
                                   isedfit=False, columns=('Z', 'LAMBDA_CHISQ'))
cat.add_columns_from(cat1)
cat

In [ ]:
mhalo = np.log10(lambda2mhalo(cat.lambda_chisq, redshift=cat.z))
mhalo

In [ ]:
bins = 35

In [ ]:
mstar_med, bin_edges, _ = binned_statistic(mhalo, cat.mstar_avg, statistic='median', bins=bins)
bin_width = (bin_edges[1] - bin_edges[0])
mhalo_med = bin_edges[1:] - bin_width/2
print(bin_width)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

colors = iter(sns.color_palette())

rich = cat.lambda_chisq > 100
ax.plot(mhalo_med, mstar_med, color='k', ls='-', lw=3, alpha=0.5)
ax.plot(mhalo_med, mstar_p75, color='k', ls='--', lw=3, alpha=0.5)
ax.plot(mhalo_med, mstar_p25, color='k', ls='--', lw=3, alpha=0.5)

g = ax.errorbar(gonz['m500'], gonz['mbcg'], yerr=gonz['mbcg_err'], color=next(colors),
                fmt='o', label='Gonzalez+13', markersize=10)
k = ax.errorbar(krav['m500'], krav['mbcg'], yerr=krav['mbcg_err'], color=next(colors),
                fmt='s', label='Kravtsov+14', markersize=10)
r = ax.scatter(mhalo[rich], cat.mstar_avg[rich], alpha=0.9, color=next(colors),
               edgecolor='k', marker='D', s=50, label=r'redMaPPer ($\lambda>100$)')

ax.text(0.12, 0.16, 'redMaPPer\n$0.1<z<0.3$', multialignment='center',
        transform=ax.transAxes, fontsize=14)
m500 = np.linspace(13.55, 15.25, 50)
ff = ax.plot(m500, np.polyval([0.33, 12.24], m500-14.5), ls='-', 
            color='k', label=r'$M_{*}\propto M_{500}^{0.33}$')
ax.text(0.12, 0.9, r'$M_{*}\propto M_{500}^{0.33}$', multialignment='center',
        transform=ax.transAxes, fontsize=16)
ax.plot([13.55, 13.68], [12.8, 12.8], ls='-', color='k') # hack!!!

#ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.xaxis.set_major_locator(MultipleLocator(0.5))

hh = [g, k, r]
ax.legend(hh, [H.get_label() for H in hh], loc='lower right',
          frameon=True, fontsize=16)
#ax.legend(ff, ff.get_label(), loc='upper left',
#          frameon=True, fontsize=16)
#ax.legend(loc='upper left', frameon=True, fontsize=16)

ax.set_ylim(10.5, 13)
ax.set_xlim(13.5, 15.3)
ax.set_xlabel(r'$\log_{10}\, (M_{500}\ /\ M_{\odot})$')
ax.set_ylabel(r'$\log_{10}\, (M_{*}\ /\ M_{\odot})$')

In [ ]:
ff

In [ ]:
stop

In [ ]:
cat

In [ ]:
legacyhalos_dir = os.getenv('LEGACYHALOS_DIR')

In [ ]:
parentfile = os.path.join(legacyhalos_dir, 'legacyhalos-parent-isedfit.fits')

In [ ]:
ls = Table(fits.getdata(parentfile, extname='LSPHOT-ISEDFIT'))
ls

In [ ]:
_ = plt.hist(ls['MSTAR_AVG'], bins=100)

In [ ]:
_ = plt.hist(sdss['MSTAR_AVG'], bins=100)

In [ ]:
_ = plt.hist(sdss['MSTAR_AVG'] - ls['MSTAR_AVG'], bins=200)
plt.xlim(-0.5, 0.5)

In [ ]:
sdss = Table(fits.getdata(parentfile, extname='SDSSPHOT-ISEDFIT'))
sdss

In [ ]:
data = np.vstack( (ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG']))
data.shape

In [ ]:
k = kde.gaussian_kde(data.T)
#xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
#zi = k(np.vstack([xi.flatten(), yi.flatten()]))

In [ ]:
fig, ax = plt.subplots()
ax.hexbin(ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG'],
          mincnt=1)

In [ ]:
sns.jointplot(ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG'], 
              kind="kde", color="#4CB391", xlim=(10, 13), ylim=(-0.5, 0.5))

In [ ]:
sns.kdeplot(ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG'], 
            cmap="Blues", shade=True, shade_lowest=True, cbar=True,
            cut=0,
           )

In [ ]:
help(sns.kdeplot)